In [1]:
import paho.mqtt.client as mqtt
import time
from datetime import datetime
import random
import json
from random import randint
import os

#mqtt_broker_host = "172.208.48.92"
mqtt_broker_host = "localhost"
mqtt_broker_port = 1883
# mqtt_topic = "sensor/location"
location_id_base = "locationUser"
location_id_counter = 1


list_users = ["A","B","C"]


def getPosMinMax():
    xA=randint(0,100)
    xB=randint(0,100)
    
    x1=min(xA,xB)
    x2=max(xA,xB)
    return x1,x2

# Función para simular el sensor binario
def simulate_binary_sensor():
    sensor_data = []

    for user_id in list_users:
        x1,x2=getPosMinMax()
        y1,y2=getPosMinMax()
        uncertain=random.uniform(0, 1)
        timestamp = datetime.now().isoformat()  # Obtén la fecha y hora actual en formato ISO

        message = f"timestamp_sensor|{timestamp}|x1|{x1}|x2|{x2}|y1|{y1}|y2|{y2}|uncertain|{uncertain}|user_id|{user_id}"
        sensor_data.append(message)

    return sensor_data

def on_publish(client, userdata, mid):
    print(f"Mensaje publicado con éxito (ID: {mid})")
    
def getTime():
    return str(int(time.time()))+"\t"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("Config Client")
# Configuración del cliente MQTT
client = mqtt.Client()
client.on_publish = on_publish

# Conexión al broker MQTT
client.connect(mqtt_broker_host, mqtt_broker_port, keepalive=60)
client.loop_start()

try:
    while True:
        # Simula la generación de un mensaje aleatorio
        sensor_data_list = simulate_binary_sensor()

        for sensor_data in sensor_data_list:
            location_id = f"{location_id_base}{location_id_counter:03d}"
            topic = f"/ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/{location_id}/attrs"

            # Publica el mensaje en el topic
            result = client.publish(topic, sensor_data, qos=2)
            print("mqrr_topic->", topic, "sensor_data->", sensor_data, "result->", result)

            location_id_counter += 1

        # Espera antes de la siguiente lectura
        time.sleep(randint(1,5))

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
    client.disconnect()

Config Client
mqrr_topic-> /ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/locationUser001/attrs sensor_data-> timestamp_sensor|2024-06-08T19:21:12.556151|x1|68|x2|84|y1|73|y2|96|uncertain|0.7622758859816617|user_id|A result-> (0, 1)
mqrr_topic-> /ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/locationUser002/attrs sensor_data-> timestamp_sensor|2024-06-08T19:21:12.556166|x1|15|x2|77|y1|8|y2|20|uncertain|0.8046579474248373|user_id|B result-> (0, 2)
mqrr_topic-> /ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/locationUser003/attrs sensor_data-> timestamp_sensor|2024-06-08T19:21:12.556172|x1|37|x2|41|y1|52|y2|99|uncertain|0.02989811871053638|user_id|C result-> (0, 3)
mqrr_topic-> /ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/locationUser004/attrs sensor_data-> timestamp_sensor|2024-06-08T19:21:14.556951|x1|17|x2|60|y1|30|y2|58|uncertain|0.7012077487903233|user_id|A result-> (4, 4)
mqrr_topic-> /ul/f1aa255e-yfzd-4253-znvy-ee3dda78a28c/locationUser005/attrs sensor_data-> timestamp_sensor|2024-06-08T19:21:14.556977|